In [1]:
%matplotlib inline
import diffimTests as dit

Using flux=600 for SNR=5 (see cell #4 of notebook '30. 4a. other psf models-real PSFs')

In [ ]:
testResults1 = dit.multi.runMultiDiffimTests(varSourceFlux=600.)
testResults2 = dit.multi.runMultiDiffimTests(varSourceFlux=600., templateNoNoise=False)
testResults3 = dit.multi.runMultiDiffimTests(varSourceFlux=600., templateNoNoise=False, skyLimited=False)

RUNNING: 100
CORES: 3


diffimTests/makeFakeImages.py:189: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  (offset2[0]-starSize+1):(offset2[0]+starSize)] += tmp
diffimTests/makeFakeImages.py:227: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  (offset2[0]-starSize+1):(offset2[0]+starSize)] += tmp


In [ ]:
import cPickle, gzip
cPickle.dump(testResults1, testResults2, testResults3, file=gzip.GzipFile('tmp_pkl.gz', 'wb'))

In [ ]:
import cPickle, gzip
testResults1, testResults2, testResults3 = cPickle.load(gzip.GzipFile('tmp_pkl.gz', 'rb'))
dit.multi.plotResults(testResults1, title='Noise-free template; sky-limited')

In [ ]:
dit.multi.plotResults(testResults1, title='Noise-free template; sky-limited', asHist=True, doPrint=False)